In [57]:
import json
import numpy as np
from scipy import sparse
import torch
import pickle
import heapq

In [2]:
question2idx = np.load('data/question2idx.npy',allow_pickle=True).item()
idx2question = np.load('data/idx2question.npy',allow_pickle=True).item()

In [26]:
model_path = torch.load('../alg/model-100/results.pth')
qq_table = sparse.load_npz('../alg/data/qq_table.npz').toarray()
qs_table = sparse.load_npz('../alg/data/qs_table.npz').toarray()
ss_table = sparse.load_npz('../alg/data/ss_table.npz').toarray()

In [70]:
from gikt import GIKT
from utils import gen_gikt_graph, build_adj_list
model_path = 'model-100/results.pth'
qq_table_path = 'data/qq_table.npz'
qs_table_path = 'data/qs_table.npz'
ss_table_path = 'data/ss_table.npz'
question2idx_path = 'data/question2idx.npy'
idx2question_path = 'data/idx2question.npy'

qq_table = sparse.load_npz(qq_table_path).toarray()
# qs_table = sparse.load_npz(qs_table_path).toarray()
qs_table = torch.tensor(sparse.load_npz(qs_table_path).toarray(), dtype=torch.int64, device='cpu')
ss_table = sparse.load_npz(ss_table_path).toarray()
question2idx = np.load(question2idx_path, allow_pickle=True).item()
idx2question = np.load(idx2question_path, allow_pickle=True).item()
num_question = torch.tensor(qs_table.shape[0], device='cpu')
num_skill = torch.tensor(qs_table.shape[1], device='cpu')
q_neighbors_list, s_neighbors_list = build_adj_list()
q_neighbors, s_neighbors = gen_gikt_graph(
    q_neighbors_list, s_neighbors_list, 4, 10)
q_neighbors = torch.tensor(q_neighbors, dtype=torch.int64, device='cpu')
s_neighbors = torch.tensor(s_neighbors, dtype=torch.int64, device='cpu')
model = GIKT(
    num_question, num_skill, q_neighbors, s_neighbors, qs_table
).to('cpu')
model.load_state_dict(torch.load(model_path))

FileNotFoundError: [Errno 2] No such file or directory: 'alg/data/qs_table.npz'

In [64]:
num = 10 # 假设推荐10

with open('test_history.json', 'r', encoding='utf-8') as f:
        json_data = json.load(f)
history_answers = json_data['data']['exerciseRecordList']  # 获取历史答题记

In [47]:
q_list = [question2idx[a['exerciseId']] for a in history_answers] # 获取答题记录的index
# 获取相关的习题
q_set_related = set()  # 所有相关问题的id集合
for q_id in q_list:
    q_set_related.update(np.where(qq_table[q_id] > 0)[0].tolist())
q_list_related = list(q_set_related)  # 所有相关问题的id数组
if num > len(q_list_related):  # 相关题目比需要推荐的少，允许[重复]选
    q_list_related *= (num / len(q_list_related) + 1)

In [65]:
len(set(q_list))

22

In [32]:
filtered_list = [item for item in q_list_related if item not in q_list]

In [33]:
len(filtered_list)

47

In [34]:
a_list = [1 if a['score'] > 0 else 0 for a in history_answers] # 获取答题记录的index

In [35]:
len(a_list)

100

In [36]:
max_length = 200  # 需要改为传值？
question_tensor_list = []
answer_tensor_list = []
mask_tensor_list = []
time_step = len(q_list)
for q_related in q_list_related:
    combined_q_list = q_list + [q_related]  # 原始的答题记录加上最后一道用于预测的题目
    combined_a_list = a_list + [0]
    combined_m_list = [1 for i in range(len(combined_q_list))]
    if len(combined_q_list) < max_length:  # 如果序列长度小于最大长度
        combined_q_list += [0] * (max_length - len(combined_q_list))
        combined_a_list += [0] * (max_length - len(combined_a_list))
        combined_m_list += [0] * (max_length - len(combined_m_list))
    elif len(combined_q_list) > max_length:
        combined_q_list = combined_q_list[-max_length:]
        combined_a_list = combined_a_list[-max_length:]
        combined_m_list = combined_m_list[-max_length:]
    question_tensor_list.append(combined_q_list)
    answer_tensor_list.append(combined_a_list)
    mask_tensor_list.append(combined_m_list)
question_tensor = torch.tensor(question_tensor_list, dtype=torch.int64)
answer_tensor = torch.tensor(answer_tensor_list, dtype=torch.int64)
mask_tensor = torch.tensor(mask_tensor_list, dtype=torch.int64) 

In [49]:
c_list = model(
        question=question_tensor,  
        response=answer_tensor,  
        mask=mask_tensor  
).squeeze(dim=0).tolist()


TypeError: 'collections.OrderedDict' object is not callable

In [ ]:

 # 模型预测
c_list = model(
    question=question_tensor,
    response=answer_tensor,
    mask=mask_tensor
).squeeze(dim=0).tolist()

# 获取预测结果
predict_list = [c_list[i][time_step] for i in range(len(q_list_related))]
# 排序预测结果
recommend = heapq.nsmallest(num, zip(predict_list, q_list_related))
# 四舍五入预测结果，把推荐问题的index转为问题id
c_list, q_list_recommend = [round(rec[0], 4) for rec in recommend], [idx2question[rec[1]] for rec in recommend]

In [73]:
# 测试协同过滤
from app.test_kt_interface import cal_stu_knowledge_state_similarity, fetch_exercise_records, get_knowledge_state_by_id
student_id = 53167
num = 10 # 推荐10个
# 获取相似学生
similar_users = cal_stu_knowledge_state_similarity(student_id)
# 存储每个题目的所有预测准确率
question_accuracy = {}
# 遍历相似学生，获取他们的knowledge state
for user_id, similarity in similar_users.items():
    knowledge_state = get_knowledge_state_by_id(user_id)
    if knowledge_state is not None :
        # 获取学生对题目的答题准确率
        history_answers = fetch_exercise_records(
            user_id)['exerciseRecordList']  # 获取历史答题记录
        q_list = [question2idx[a['exerciseId']]
                for a in history_answers]  # 获取答题记录的index
        a_list = [1 if a['score'] >
              0 else 0 for a in history_answers]  # 获取答题answer记录
        # m_list =
        question_tensor = torch.tensor(q_list, dtype=torch.int64) 
        answer_tensor = torch.tensor(a_list, dtype=torch.int64) 
        # mask_tensor =torch.tensor(m_list, dtype=torch.int64)
        # 模型预测
        c_list = model(
            question=question_tensor,
            response=answer_tensor,
            mask=mask_tensor
        ).squeeze(dim=0).tolist()
         # 记录每个题目的预测准确率
        for q_id, accuracy in zip(q_list, c_list):
            if q_id != -1:  # 确保 q_id 是有效的
                if q_id not in question_accuracy:
                    question_accuracy[q_id] = []
                question_accuracy[q_id].append(accuracy)
        
# 返回前K个（根据预测的答题准确率又低到高得到推荐前K道题目）
# 计算每个题目的平均准确率
question_average_accuracy = {q_id: np.mean(accuracies) for q_id, accuracies in question_accuracy.items()}

# 输出每个题目的平均准确率
for q_id, avg_acc in question_average_accuracy.items():
    print(f"题目 ID: {q_id}, 平均准确率: {avg_acc:.2f}")

# 升序排序
sorted_questions = sorted(question_average_accuracy.items(), key=lambda item: item[1])

# 提取前 n 个题目的平均准确率
top_n_questions = sorted_questions[:num]


ModuleNotFoundError: No module named 'app.module_one'

In [71]:
c_list[0]

NameError: name 'c_list' is not defined

In [15]:
c_list[2][time_step]

0.9990842342376709

In [56]:
len(q_list_related)

68

In [67]:
len(q_list)

100

In [34]:
frequency = {}
for item in q_list:
    if frequency.get(item):
        frequency[item] = frequency[item]+1
    else:
        frequency[item] = 1
for item in filtered_list:
    frequency[item] = 0

In [76]:
len(set(q_list))

22

In [77]:
len(q_list_related)

68

In [78]:
len(filtered_list)

46

In [86]:
len(frequency)

68

In [35]:
predict = []
for i in range(len(q_list_related)):
    predict.append({
        'exerciseId': idx2question[q_list_related[i]],
        'predict': c_list[i][time_step-1],
        'isInHistory': 0 if q_list_related[i] in filtered_list else 1,
        'frequency': frequency[q_list_related[i]]
    })

In [64]:
len(predict)

68

In [103]:
len(q_list)

100

In [41]:
predict_list = [c_list[i][time_step] for i in range(len(q_list_related))]

In [19]:
len(predict_list)

69

In [42]:
recommend = heapq.nsmallest(num, zip(predict_list, q_list_related))
recommend

[(3.94132426348448e-12, 15),
 (4.937236036672843e-11, 136),
 (1.7563205290116457e-07, 127),
 (2.0057963467934314e-07, 1),
 (9.080370659830805e-07, 59),
 (9.591861953595071e-07, 171),
 (4.34222056355793e-06, 122),
 (1.0386169378762133e-05, 164),
 (1.3969324754725676e-05, 9),
 (5.104697265778668e-05, 144)]

In [56]:

c_list, q_list_recommend = [round(rec[0], 4) for rec in recommend], [idx2question[rec[1]] for rec in recommend]

In [57]:
q_list_recommend

[126, 1173, 1164, 112, 170, 1208, 1159, 1201, 120, 1181]

In [36]:
import pandas as pd
df = pd.DataFrame(predict, columns=['exerciseId', 'predict','isInHistory','frequency'])
df.to_excel('test_predict_with_answer_new1_timesep-1.xlsx')

In [49]:
s_set = set()  # 所有相关技能id集合
for q_id in q_list_recommend: # 这里为什么只更新推荐习题的相关知识点
    s_set.update(np.where(qs_table[q_id] > 0)[0].tolist())

In [53]:
qs_table.shape

(429, 145)

In [51]:
skill2idx = np.load('data/skill2idx.npy',allow_pickle=True).item()
idx2skill = np.load('data/skill2idx.npy',allow_pickle=True).item()

In [52]:
s_set


{5, 12, 13, 17, 18, 22, 29, 40, 46, 85, 95, 121}

In [ ]:
s_list = list(s_set)  # 相关技能的id数组
s_names = [s.name for s in Skill.query.filter(Skill.id.in_(s_list)).all()]  # 技能名称数组

In [ ]:

    # 计算推荐习题中技能的分布情况
   
    s_list = list(s_set)  # 相关技能的id数组
    ic(s_list)
    s_names = [s.name for s in Skill.query.filter(Skill.id.in_(s_list)).all()]  # 技能名称数组
    ic(s_names)
    s_q_num = [0 for _ in range(len(s_list))]  # 每个技能涉及的问题数量
    for q_id in q_list_recommend:
        s_list1 = np.where(qs_table[convert2Index(q_id)] > 0)[0].tolist()
        for s_id in s_list1:
            s_q_num[s_list.index(s_id)] += 1
    ic(s_q_num)
    s_values = [num / sum(s_q_num) for num in s_q_num]  # 除以和，使其元素之和为1
    return {
        'data': {
            'qList': q_list_recommend,
            'cList': c_list,
            'skillData': [{
                'value': value,
                'name': name
            } for name, value in zip(s_names, s_values)]
        }
    }
